In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Congestive_heart_failure"
cohort = "GSE93101"

# Input paths
in_trait_dir = "../../input/GEO/Congestive_heart_failure"
in_cohort_dir = "../../input/GEO/Congestive_heart_failure/GSE93101"

# Output paths
out_data_file = "../../output/preprocess/Congestive_heart_failure/GSE93101.csv"
out_gene_data_file = "../../output/preprocess/Congestive_heart_failure/gene_data/GSE93101.csv"
out_clinical_data_file = "../../output/preprocess/Congestive_heart_failure/clinical_data/GSE93101.csv"
json_path = "../../output/preprocess/Congestive_heart_failure/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Molecular Prognosis of Cardiogenic Shock Patients under Extracorporeal Membrane Oxygenation"
!Series_summary	"Prognosis for cardiogenic shock patients under ECMO was our study goal. Success defined as survived more than 7 days after ECMO installation and failure died or had multiple organ failure in 7 days. Total 34 cases were enrolled, 17 success and 17 failure."
!Series_summary	"Peripheral blood mononuclear cells collected at ECMO installation were used analyzed."
!Series_overall_design	"Analysis of the cardiogenic shock patients at extracorporeal membrane oxygenation treatment by genome-wide expression and methylation. Transcriptomic profiling and DNA methylation between successful and failure groups were analyzed."
!Series_overall_design	"This submission represents the transcriptome data."
Sample Characteristics Dictionary:
{0: ['course: Acute myocarditis', 'course: Acute myocardial infarction', 'course: Dilated cardiomyopathy, DCMP', 'course:

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains transcriptome data
# "This submission represents the transcriptome data."
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability

# Trait: Congestive heart failure
# Looking at the sample characteristics, key 0 contains "course: Congestive heart failure"
# This suggests patients have different conditions, and we're interested in those with CHF
trait_row = 0

# Age: Available in key 1
age_row = 1

# Gender: Available in key 2
gender_row = 2

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait value to binary (0 or 1)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        condition = value.split(":", 1)[1].strip()
    else:
        condition = value.strip()
    
    # Check if the condition is congestive heart failure (case insensitive)
    if condition.lower() == "congestive heart failure":
        return 1
    else:
        return 0

def convert_age(value):
    """Convert age value to continuous (float)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        age_str = value.split(":", 1)[1].strip()
    else:
        age_str = value.strip()
    
    try:
        return float(age_str)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)"""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        gender = value.split(":", 1)[1].strip()
    else:
        gender = value.strip()
    
    if gender.upper() == "F":
        return 0
    elif gender.upper() == "M":
        return 1
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info for initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait_row is not None)
if trait_row is not None:
    # Extract clinical features
    clinical_features_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features_df)
    print("Preview of clinical features:")
    print(preview)
    
    # Save clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical features:
{'GSM2443799': [0.0, 33.4, 0.0], 'GSM2443800': [0.0, 51.2, 1.0], 'GSM2443801': [0.0, 51.9, 0.0], 'GSM2443802': [0.0, 47.8, 1.0], 'GSM2443803': [0.0, 41.5, 0.0], 'GSM2443804': [0.0, 67.3, 1.0], 'GSM2443805': [0.0, 52.8, 1.0], 'GSM2443806': [0.0, 16.1, 1.0], 'GSM2443807': [0.0, 78.9, 1.0], 'GSM2443808': [0.0, 53.2, 1.0], 'GSM2443809': [0.0, 70.9, 1.0], 'GSM2443810': [0.0, 59.9, 1.0], 'GSM2443811': [0.0, 21.9, 0.0], 'GSM2443812': [1.0, 45.2, 0.0], 'GSM2443813': [0.0, 52.4, 1.0], 'GSM2443814': [0.0, 32.3, 1.0], 'GSM2443815': [0.0, 52.8, 1.0], 'GSM2443816': [0.0, 55.8, 1.0], 'GSM2443817': [0.0, 47.0, 1.0], 'GSM2443818': [0.0, 55.8, 1.0], 'GSM2443819': [0.0, 57.3, 0.0], 'GSM2443820': [0.0, 31.7, 0.0], 'GSM2443821': [0.0, 49.3, 1.0], 'GSM2443822': [1.0, 66.1, 1.0], 'GSM2443823': [0.0, 55.9, 1.0], 'GSM2443824': [0.0, 49.1, 0.0], 'GSM2443825': [0.0, 63.0, 1.0], 'GSM2443826': [0.0, 21.0, 1.0], 'GSM2443827': [0.0, 53.6, 1.0], 'GSM2443828': [0.0, 50.1, 0.0], 'GSM24438

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Congestive_heart_failure/GSE93101/GSE93101_series_matrix.txt.gz
Gene data shape: (29363, 33)
First 20 gene/probe identifiers:
Index(['ILMN_1343291', 'ILMN_1651209', 'ILMN_1651228', 'ILMN_1651229',
       'ILMN_1651235', 'ILMN_1651236', 'ILMN_1651237', 'ILMN_1651238',
       'ILMN_1651254', 'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268',
       'ILMN_1651278', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286',
       'ILMN_1651292', 'ILMN_1651303', 'ILMN_1651309', 'ILMN_1651315'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# The identifiers starting with "ILMN_" are Illumina probe IDs, not human gene symbols.
# These are specific to Illumina microarray platforms and need to be mapped to standard gene symbols.
# ILMN_ prefix indicates Illumina's proprietary probe identifiers from their microarray platforms.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Check for gene information in the SPOT_ID.1 column which appears to contain gene names
print("\nAnalyzing SPOT_ID.1 column for gene symbols:")
if 'SPOT_ID.1' in gene_annotation.columns:
    # Extract a few sample values
    sample_values = gene_annotation['SPOT_ID.1'].head(3).tolist()
    for i, value in enumerate(sample_values):
        print(f"Sample {i+1} excerpt: {value[:200]}...")  # Print first 200 chars
        # Test the extract_human_gene_symbols function on these values
        symbols = extract_human_gene_symbols(value)
        print(f"  Extracted gene symbols: {symbols}")

# Try to find the probe IDs in the gene annotation
gene_data_id_prefix = gene_data.index[0].split('_')[0]  # Get prefix of first gene ID
print(f"\nGene data ID prefix: {gene_data_id_prefix}")

# Look for columns that might match the gene data IDs
for col in gene_annotation.columns:
    if gene_annotation[col].astype(str).str.contains(gene_data_id_prefix).any():
        print(f"Column '{col}' contains values matching gene data ID pattern")

# Check if there's any column that might contain transcript or gene IDs
print("\nChecking for columns containing transcript or gene related terms:")
for col in gene_annotation.columns:
    if any(term in col.upper() for term in ['GENE', 'TRANSCRIPT', 'SYMBOL', 'NAME', 'DESCRIPTION']):
        print(f"Column '{col}' may contain gene-related information")
        # Show sample values
        print(f"Sample values: {gene_annotation[col].head(3).tolist()}")



Gene annotation preview:
Columns in gene annotation: ['ID', 'Transcript', 'Species', 'Source', 'Search_Key', 'ILMN_Gene', 'Source_Reference_ID', 'RefSeq_ID', 'Entrez_Gene_ID', 'GI', 'Accession', 'Symbol', 'Protein_Product', 'Array_Address_Id', 'Probe_Type', 'Probe_Start', 'SEQUENCE', 'Chromosome', 'Probe_Chr_Orientation', 'Probe_Coordinates', 'Cytoband', 'Definition', 'Ontology_Component', 'Ontology_Process', 'Ontology_Function', 'Synonyms', 'Obsolete_Probe_Id', 'GB_ACC']
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Transcript': ['ILMN_333737', 'ILMN_333646', 'ILMN_333584', 'ILMN_333628', 'ILMN_333719'], 'Species': ['ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls', 'ILMN Controls'], 'Source': ['ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls', 'ILMN_Controls'], 'Search_Key': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144'], 'ILMN_Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC

Column 'Transcript' contains values matching gene data ID pattern


Column 'Species' contains values matching gene data ID pattern


Column 'Source' contains values matching gene data ID pattern



Checking for columns containing transcript or gene related terms:
Column 'Transcript' may contain gene-related information
Sample values: ['ILMN_333737', 'ILMN_333646', 'ILMN_333584']
Column 'ILMN_Gene' may contain gene-related information
Sample values: ['ERCC-00162', 'ERCC-00071', 'ERCC-00009']
Column 'Entrez_Gene_ID' may contain gene-related information
Sample values: [nan, nan, nan]
Column 'Symbol' may contain gene-related information
Sample values: ['ERCC-00162', 'ERCC-00071', 'ERCC-00009']


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the preview, we can see that:
# - 'ID' column in the gene annotation contains the same ILMN_ identifiers used in gene expression data
# - 'Symbol' column contains gene symbols we need to map to

# 2. Get gene mapping dataframe using the function from the library
gene_mapping = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# Examine the mapping to verify it has the expected format
print("\nGene mapping preview:")
print(f"Gene mapping shape: {gene_mapping.shape}")
print(preview_df(gene_mapping, n=5))

# 3. Apply gene mapping to convert probe-level expression to gene-level expression
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Preview the resulting gene expression data
print("\nGene expression data after mapping:")
print(f"Shape: {gene_data.shape}")
print(f"First 5 gene symbols: {gene_data.index[:5].tolist()}")

# Examine number of unique gene symbols
print(f"Number of unique gene symbols: {len(gene_data.index.unique())}")

# Check if standard gene symbols are present by looking for common genes
common_genes = ["TP53", "BRCA1", "EGFR", "TNF", "IL6"]
found_genes = [gene for gene in common_genes if gene in gene_data.index]
print(f"Common genes found: {found_genes}")



Gene mapping preview:
Gene mapping shape: (29377, 2)
{'ID': ['ILMN_3166687', 'ILMN_3165566', 'ILMN_3164811', 'ILMN_3165363', 'ILMN_3166511'], 'Gene': ['ERCC-00162', 'ERCC-00071', 'ERCC-00009', 'ERCC-00053', 'ERCC-00144']}

Gene expression data after mapping:
Shape: (20206, 33)
First 5 gene symbols: ['A1BG', 'A1CF', 'A26C3', 'A2BP1', 'A2LD1']
Number of unique gene symbols: 20206
Common genes found: ['TP53', 'BRCA1', 'EGFR', 'TNF', 'IL6']


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        # Check if trait is biased
        trait_type = 'binary' if len(linked_data[trait].unique()) <= 2 else 'continuous'
        if trait_type == "binary":
            is_biased = judge_binary_variable_biased(linked_data, trait)
        else:
            is_biased = judge_continuous_variable_biased(linked_data, trait)
            
        # Remove biased demographic features
        if "Age" in linked_data.columns:
            age_biased = judge_continuous_variable_biased(linked_data, 'Age')
            if age_biased:
                linked_data = linked_data.drop(columns='Age')
                
        if "Gender" in linked_data.columns:
            gender_biased = judge_binary_variable_biased(linked_data, 'Gender')
            if gender_biased:
                linked_data = linked_data.drop(columns='Gender')
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (20206, 33)
Gene data shape after normalization: (19445, 33)


Normalized gene expression data saved to ../../output/preprocess/Congestive_heart_failure/gene_data/GSE93101.csv
Original clinical data preview:
         !Sample_geo_accession                 GSM2443799  \
0  !Sample_characteristics_ch1  course: Acute myocarditis   
1  !Sample_characteristics_ch1                  age: 33.4   
2  !Sample_characteristics_ch1                  gender: F   
3  !Sample_characteristics_ch1           outcome: Success   

                  GSM2443800                 GSM2443801  \
0  course: Acute myocarditis  course: Acute myocarditis   
1                  age: 51.2                  age: 51.9   
2                  gender: M                  gender: F   
3           outcome: Success           outcome: Failure   

                            GSM2443802                 GSM2443803  \
0  course: Acute myocardial infarction  course: Acute myocarditis   
1                            age: 47.8                  age: 41.5   
2                            gender: M        

Data shape after handling missing values: (33, 19448)
For the feature 'Congestive_heart_failure', the least common label is '1.0' with 3 occurrences. This represents 9.09% of the dataset.
Quartiles for 'Age':
  25%: 45.2
  50% (Median): 52.4
  75%: 56.5
Min: 16.1
Max: 78.9
For the feature 'Gender', the least common label is '0.0' with 10 occurrences. This represents 30.30% of the dataset.
Dataset is not usable for analysis. No linked data file saved.
